In [1]:
import requests
import json
import pandas as pd
import numpy as np

In [2]:
filepath = '/Users/jacksonpeoples/Desktop/Springboard/CapstoneTwo/Data/Vision_Zero_Crash_Report_Data.csv'
crash_data = pd.read_csv(filepath)

weather_path = '/Users/jacksonpeoples/Desktop/Springboard/CapstoneTwo/Data/WeatherAustin.csv'
weather_data = pd.read_csv(weather_path)

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#Checking out some basic info on the dataset:
crash_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161824 entries, 0 to 161823
Data columns (total 51 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   crash_id                            161824 non-null  int64  
 1   crash_fatal_fl                      161824 non-null  object 
 2   crash_date                          161824 non-null  object 
 3   crash_time                          161824 non-null  object 
 4   case_id                             159592 non-null  object 
 5   rpt_latitude                        5349 non-null    float64
 6   rpt_longitude                       5349 non-null    float64
 7   rpt_block_num                       140411 non-null  object 
 8   rpt_street_pfx                      90518 non-null   object 
 9   rpt_street_name                     161818 non-null  object 
 10  rpt_street_sfx                      118702 non-null  object 
 11  crash_speed_limit         

In [4]:
#Quite a few columns. Many will be useful, some won't. Some have very few values and some convey the same information
#in a different form.
crash_data.head()

,crash_id,crash_fatal_fl,crash_date,crash_time,case_id,rpt_latitude,rpt_longitude,rpt_block_num,rpt_street_pfx,rpt_street_name,...,bicycle_death_count,bicycle_serious_injury_count,pedestrian_death_count,pedestrian_serious_injury_count,motorcycle_death_count,motorcycle_serious_injury_count,other_death_count,other_serious_injury_count,onsys_fl,private_dr_fl
0,16776868,N,12/04/2018 03:36:00 PM,15:36:00,183381097,NaN,NaN,1900,N,IH 35 SB,...,0,0,0,0,0,0,0,0,Y,N
1,16786471,N,12/07/2018 01:38:00 PM,13:38:00,183410837,NaN,NaN,12400,NaN,N IH 35 SVRD SB,...,0,0,0,0,0,0,0,0,Y,N
2,16757244,N,11/18/2018 04:10:00 AM,04:10:00,183220392,NaN,NaN,4500,N,N IH 35 NB,...,0,0,0,0,0,0,0,0,Y,N
3,16776852,N,12/03/2018 11:19:00 AM,11:19:00,183370654,NaN,NaN,200,NaN,NUECES ST,...,0,0,0,0,0,0,0,0,N,N
4,16765750,N,11/29/2018 09:07:00 AM,09:07:00,183330494,NaN,NaN,4100,N,LAMAR,...,0,0,0,0,0,0,0,0,Y,N


In [5]:
#Will go ahead and get rid of columns that pertain to police response (not helpful in predicting danger) and
#a few that are almost entirely empty.
del_col = ['case_id', 'rpt_latitude', 'rpt_longitude', 'contrib_factr_p1_id', 'contrib_factr_p2_id', 'units_involved','atd_mode_category_metadata', 'street_nbr_2']
crash_data = crash_data.drop(del_col, 1)

In [6]:
#Another question worth asking is should I regard fatality info? Use it to weight the danger of a given crash,
#or should I drop the data and focus only on the likelihood of a crash period?
crash_data['crash_fatal_fl'].value_counts()

N    161035
Y       789
Name: crash_fatal_fl, dtype: int64

In [7]:
sample = crash_data['crash_date'][7]
print(sample)
print(type(sample))

11/21/2018 05:39:00 PM
<class 'str'>


In [8]:
#Would like to get rid of the time info in 'crash_date' so that I can work with date/time of day
#independently.
def split_time(crash_date):
    tup = crash_date.partition(' ')
    return tup[0]
print(split_time(sample))

11/21/2018


In [9]:
#Will now apply the function to the 'crash_date' column:
crash_data['crash_date_only'] = crash_data['crash_date'].apply(split_time)

In [10]:
#See if it worked:
crash_data.head()

,crash_id,crash_fatal_fl,crash_date,crash_time,rpt_block_num,rpt_street_pfx,rpt_street_name,rpt_street_sfx,crash_speed_limit,road_constr_zone_fl,...,bicycle_serious_injury_count,pedestrian_death_count,pedestrian_serious_injury_count,motorcycle_death_count,motorcycle_serious_injury_count,other_death_count,other_serious_injury_count,onsys_fl,private_dr_fl,crash_date_only
0,16776868,N,12/04/2018 03:36:00 PM,15:36:00,1900,N,IH 35 SB,NaN,-1.0,N,...,0,0,0,0,0,0,0,Y,N,12/04/2018
1,16786471,N,12/07/2018 01:38:00 PM,13:38:00,12400,NaN,N IH 35 SVRD SB,NaN,70.0,N,...,0,0,0,0,0,0,0,Y,N,12/07/2018
2,16757244,N,11/18/2018 04:10:00 AM,04:10:00,4500,N,N IH 35 NB,NaN,70.0,N,...,0,0,0,0,0,0,0,Y,N,11/18/2018
3,16776852,N,12/03/2018 11:19:00 AM,11:19:00,200,NaN,NUECES ST,NaN,30.0,N,...,0,0,0,0,0,0,0,N,N,12/03/2018
4,16765750,N,11/29/2018 09:07:00 AM,09:07:00,4100,N,LAMAR,BLVD,-1.0,N,...,0,0,0,0,0,0,0,Y,N,11/29/2018


In [11]:
print(type(crash_data['crash_date_only'][3]))
print(type(crash_data['crash_time'][3]))

<class 'str'>
<class 'str'>


In [12]:
#Need to convert both 'crash_date' and 'crash_time' to timeseries types.
crash_data['crash_date_only']= pd.to_datetime(crash_data['crash_date_only'])
crash_data['crash_date']= pd.to_datetime(crash_data['crash_date'])
crash_data['crash_time']= pd.to_datetime(crash_data['crash_time'])
crash_data['crash_time'] = [time.time() for time in crash_data['crash_time']]

In [13]:
print(type(crash_data['crash_date'][3]))
print(type(crash_data['crash_time'][3]))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'datetime.time'>


In [14]:
crash_data.head()

,crash_id,crash_fatal_fl,crash_date,crash_time,rpt_block_num,rpt_street_pfx,rpt_street_name,rpt_street_sfx,crash_speed_limit,road_constr_zone_fl,...,bicycle_serious_injury_count,pedestrian_death_count,pedestrian_serious_injury_count,motorcycle_death_count,motorcycle_serious_injury_count,other_death_count,other_serious_injury_count,onsys_fl,private_dr_fl,crash_date_only
0,16776868,N,2018-12-04 15:36:00,15:36:00,1900,N,IH 35 SB,NaN,-1.0,N,...,0,0,0,0,0,0,0,Y,N,2018-12-04
1,16786471,N,2018-12-07 13:38:00,13:38:00,12400,NaN,N IH 35 SVRD SB,NaN,70.0,N,...,0,0,0,0,0,0,0,Y,N,2018-12-07
2,16757244,N,2018-11-18 04:10:00,04:10:00,4500,N,N IH 35 NB,NaN,70.0,N,...,0,0,0,0,0,0,0,Y,N,2018-11-18
3,16776852,N,2018-12-03 11:19:00,11:19:00,200,NaN,NUECES ST,NaN,30.0,N,...,0,0,0,0,0,0,0,N,N,2018-12-03
4,16765750,N,2018-11-29 09:07:00,09:07:00,4100,N,LAMAR,BLVD,-1.0,N,...,0,0,0,0,0,0,0,Y,N,2018-11-29


In [15]:
#create a function to get the weekday for each row based on the date.
def get_weekday(date):
    return date.weekday()

#use that function to create weekday column.
crash_data['Weekday'] = crash_data['crash_date'].apply(get_weekday)

In [16]:
crash_data['Weekday'].value_counts()

4    26897
3    24202
1    23605
2    23297
5    22844
0    21798
6    19181
Name: Weekday, dtype: int64

In [17]:
crash_data['crash_date'].value_counts()

2016-07-18 17:05:00    4
2019-10-18 09:00:00    4
2013-04-04 17:30:00    3
2014-01-18 14:30:00    3
2018-10-08 13:43:00    3
                      ..
2018-07-03 06:32:00    1
2013-06-28 22:35:00    1
2019-09-07 10:13:00    1
2017-05-14 14:47:00    1
2017-12-31 22:00:00    1
Name: crash_date, Length: 158083, dtype: int64

In [18]:
print(crash_data['crash_date'].max())
print(crash_data['crash_date'].min())

2020-08-31 23:38:00
2010-01-01 00:59:00


In [19]:
print(type(crash_data['crash_date'][3]))
print(type(crash_data['crash_time'][3]))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'datetime.time'>


In [20]:
#need to create a column in the crash data that rounds the time to the nearest hour because the weather data is 
#hourly.
def rounding(time):
    return time.floor('H')

crash_data['rounded_time'] = crash_data['crash_date'].apply(rounding)

In [21]:
crash_data['rounded_time'].head()

0   2018-12-04 15:00:00
1   2018-12-07 13:00:00
2   2018-11-18 04:00:00
3   2018-12-03 11:00:00
4   2018-11-29 09:00:00
Name: rounded_time, dtype: datetime64[ns]

In [22]:
weather_data.head()

,dt,dt_iso,timezone,city_name,lat,lon,temp,feels_like,temp_min,temp_max,...,wind_deg,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,1199145600,2008-01-01 00:00:00 +0000 UTC,-21600,Austin,30.267153,-97.743061,57.49,46.69,55.74,57.92,...,360,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01n
1,1199149200,2008-01-01 01:00:00 +0000 UTC,-21600,Austin,30.267153,-97.743061,55.13,47.39,53.96,57.22,...,360,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01n
2,1199152800,2008-01-01 02:00:00 +0000 UTC,-21600,Austin,30.267153,-97.743061,52.79,42.84,51.08,54.89,...,10,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01n
3,1199156400,2008-01-01 03:00:00 +0000 UTC,-21600,Austin,30.267153,-97.743061,50.65,38.66,48.98,51.98,...,10,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01n
4,1199160000,2008-01-01 04:00:00 +0000 UTC,-21600,Austin,30.267153,-97.743061,49.14,39.16,47.69,50.00,...,10,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01n


In [23]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115449 entries, 0 to 115448
Data columns (total 25 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   dt                   115449 non-null  int64  
 1   dt_iso               115449 non-null  object 
 2   timezone             115449 non-null  int64  
 3   city_name            115449 non-null  object 
 4   lat                  115449 non-null  float64
 5   lon                  115449 non-null  float64
 6   temp                 115449 non-null  float64
 7   feels_like           115449 non-null  float64
 8   temp_min             115449 non-null  float64
 9   temp_max             115449 non-null  float64
 10  pressure             115449 non-null  int64  
 11  sea_level            0 non-null       float64
 12  grnd_level           0 non-null       float64
 13  humidity             115449 non-null  int64  
 14  wind_speed           115449 non-null  float64
 15  wind_deg         

In [24]:
def del_tz(text):
    tup = text.partition(' +')
    return tup[0]

def to_dt(time_string):
    return pd.to_datetime(time_string)

def cst_conv(time):
    return time - np.timedelta64(5, 'h')

def time_clean(time_string):
    return cst_conv(to_dt(del_tz(time_string)))


In [25]:
weather_data['rounded_time'] = weather_data['dt_iso'].apply(time_clean)

In [51]:
print(weather_data['rounded_time'][4])
print(type(weather_data['rounded_time'][4]))
print(crash_data['rounded_time'][4])
print(type(crash_data['rounded_time'][4]))
print(crash_data['rounded_time'].max())
print(crash_data['rounded_time'].min())
print(weather_data['rounded_time'].max())
print(weather_data['rounded_time'].min())

2007-12-31 23:00:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
2018-11-29 09:00:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
2020-08-31 23:00:00
2010-01-01 00:00:00
2020-10-18 18:00:00
2007-12-31 19:00:00


In [100]:
print(weather_data['rounded_time'].nunique())
print(weather_data['dt_iso'].nunique())
print(weather_data.groupby(['dt_iso','timezone']).ngroups)

112200
112200
112200


In [107]:
for col in weather_data.columns:
    if weather_data['dt_iso'].nunique() < weather_data.groupby(['dt_iso',col]).ngroups:
        print(col)
        print(weather_data.groupby(['dt_iso',col]).ngroups)

weather_id
115449
weather_main
115449
weather_description
115449
weather_icon
115443


In [101]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115449 entries, 0 to 115448
Data columns (total 26 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   dt                   115449 non-null  int64         
 1   dt_iso               115449 non-null  object        
 2   timezone             115449 non-null  int64         
 3   city_name            115449 non-null  object        
 4   lat                  115449 non-null  float64       
 5   lon                  115449 non-null  float64       
 6   temp                 115449 non-null  float64       
 7   feels_like           115449 non-null  float64       
 8   temp_min             115449 non-null  float64       
 9   temp_max             115449 non-null  float64       
 10  pressure             115449 non-null  int64         
 11  sea_level            0 non-null       float64       
 12  grnd_level           0 non-null       float64       
 13  humidity      

In [54]:
crash_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161824 entries, 0 to 161823
Data columns (total 46 columns):
 #   Column                              Non-Null Count   Dtype         
---  ------                              --------------   -----         
 0   crash_id                            161824 non-null  int64         
 1   crash_fatal_fl                      161824 non-null  object        
 2   crash_date                          161824 non-null  datetime64[ns]
 3   crash_time                          161824 non-null  object        
 4   rpt_block_num                       140411 non-null  object        
 5   rpt_street_pfx                      90518 non-null   object        
 6   rpt_street_name                     161818 non-null  object        
 7   rpt_street_sfx                      118702 non-null  object        
 8   crash_speed_limit                   161819 non-null  float64       
 9   road_constr_zone_fl                 161819 non-null  object        
 10  latitude

In [40]:
df = pd.merge(crash_data, weather_data, how= 'left', on = 'rounded_time')

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 169404 entries, 0 to 169403
Data columns (total 71 columns):
 #   Column                              Non-Null Count   Dtype         
---  ------                              --------------   -----         
 0   crash_id                            169404 non-null  int64         
 1   crash_fatal_fl                      169404 non-null  object        
 2   crash_date                          169404 non-null  datetime64[ns]
 3   crash_time                          169404 non-null  object        
 4   rpt_block_num                       147029 non-null  object        
 5   rpt_street_pfx                      94899 non-null   object        
 6   rpt_street_name                     169398 non-null  object        
 7   rpt_street_sfx                      124033 non-null  object        
 8   crash_speed_limit                   169399 non-null  float64       
 9   road_constr_zone_fl                 169399 non-null  object        
 10  latitude

In [56]:
#Very confused why there are extra rows...
print(crash_data['crash_id'].nunique())
print(df['crash_id'].nunique())

161824
161824


In [58]:
df['crash_id'].value_counts()

17552596    3
15554103    3
13736215    3
14815982    3
15111518    3
           ..
13216737    1
16532215    1
13490863    1
15604897    1
15204352    1
Name: crash_id, Length: 161824, dtype: int64

In [75]:
df['crash_id'].value_counts().value_counts()

1    155103
2      5862
3       859
Name: crash_id, dtype: int64

In [62]:
df.loc[df['crash_id'] == 17552596]['rounded_time']

30846   2020-02-06 01:00:00
30847   2020-02-06 01:00:00
30848   2020-02-06 01:00:00
Name: rounded_time, dtype: datetime64[ns]

In [71]:
repeat_time = df['rounded_time'][30846]
weather_data.loc[weather_data['rounded_time'] == repeat_time]

,dt,dt_iso,timezone,city_name,lat,lon,temp,feels_like,temp_min,temp_max,...,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon,rounded_time
109023,1580968800,2020-02-06 06:00:00 +0000 UTC,-21600,Austin,30.267153,-97.743061,32.41,24.89,30.99,34.0,...,0.51,NaN,0.33,NaN,90,500,Rain,light rain,10n,2020-02-06 01:00:00
109024,1580968800,2020-02-06 06:00:00 +0000 UTC,-21600,Austin,30.267153,-97.743061,32.41,24.89,30.99,34.0,...,0.51,NaN,0.33,NaN,90,600,Snow,light snow,13n,2020-02-06 01:00:00
109025,1580968800,2020-02-06 06:00:00 +0000 UTC,-21600,Austin,30.267153,-97.743061,32.41,24.89,30.99,34.0,...,0.51,NaN,0.33,NaN,90,701,Mist,mist,50n,2020-02-06 01:00:00


In [66]:
#seems like there were repeats in rounded_time in the weather data.
print(len(weather_data['dt_iso']))
print(weather_data['dt_iso'].nunique())

115449
112200


In [68]:
print(len(weather_data['lon']))
print(weather_data['lon'].nunique())

115449
1


In [78]:
weath_date_vc = weather_data['rounded_time'].value_counts()

In [85]:
repeat_3 = weath_date_vc.loc[weath_date_vc == 3]

In [94]:
print(repeat_3.index.min())
print(repeat_3.index.max())

2008-03-06 15:00:00
2020-09-09 07:00:00


In [91]:
weather_data.loc[weather_data['rounded_time'].isin(repeat_3).index]

,dt,dt_iso,timezone,city_name,lat,lon,temp,feels_like,temp_min,temp_max,...,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon,rounded_time
0,1199145600,2008-01-01 00:00:00 +0000 UTC,-21600,Austin,30.267153,-97.743061,57.49,46.69,55.74,57.92,...,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01n,2007-12-31 19:00:00
1,1199149200,2008-01-01 01:00:00 +0000 UTC,-21600,Austin,30.267153,-97.743061,55.13,47.39,53.96,57.22,...,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01n,2007-12-31 20:00:00
2,1199152800,2008-01-01 02:00:00 +0000 UTC,-21600,Austin,30.267153,-97.743061,52.79,42.84,51.08,54.89,...,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01n,2007-12-31 21:00:00
3,1199156400,2008-01-01 03:00:00 +0000 UTC,-21600,Austin,30.267153,-97.743061,50.65,38.66,48.98,51.98,...,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01n,2007-12-31 22:00:00
4,1199160000,2008-01-01 04:00:00 +0000 UTC,-21600,Austin,30.267153,-97.743061,49.14,39.16,47.69,50.00,...,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01n,2007-12-31 23:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115444,1603047600,2020-10-18 19:00:00 +0000 UTC,-18000,Austin,30.267153,-97.743061,87.19,86.67,84.00,90.00,...,1.02,NaN,NaN,NaN,1,501,Rain,moderate rain,10d,2020-10-18 14:00:00
115445,1603051200,2020-10-18 20:00:00 +0000 UTC,-18000,Austin,30.267153,-97.743061,88.79,87.03,87.01,91.00,...,0.51,NaN,NaN,NaN,20,500,Rain,light rain,10d,2020-10-18 15:00:00
115446,1603054800,2020-10-18 21:00:00 +0000 UTC,-18000,Austin,30.267153,-97.743061,89.67,87.60,88.00,91.40,...,NaN,NaN,NaN,NaN,20,801,Clouds,few clouds,02d,2020-10-18 16:00:00
115447,1603058400,2020-10-18 22:00:00 +0000 UTC,-18000,Austin,30.267153,-97.743061,89.22,83.39,88.00,91.00,...,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01d,2020-10-18 17:00:00


In [108]:
crash_data['crash_date'].min()

Timestamp('2010-01-01 00:59:00')

In [109]:
crash_data['crash_date'].max()

Timestamp('2020-08-31 23:38:00')